<a href="https://colab.research.google.com/github/yumnahussain444/PREDOC-Coding-Sample/blob/main/Python_Sample_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ------------------------------------------------------------------------------
# Phillips Curve Analysis: Inflation vs. Unemployment in the U.S. Economy
#
# This script investigates the empirical relationship between unemployment and
# inflation in the United States, with a focus on the Phillips Curve hypothesis.
# The analysis uses monthly data from the Federal Reserve Economic Data (FRED)
# database and focuses on the post-1950 period to capture modern macroeconomic
# dynamics.
#
# The analysis performs the following tasks:
# - Visualizes time series behavior of unemployment and inflation using
#   dual-axis line plots.
# - Estimates the static Phillips Curve relationship via scatter plots and
#   OLS regression trendlines.
# - Computes five-year rolling correlations to evaluate how the relationship
#   evolves over time.
# - Disaggregates the Phillips Curve by decade to examine structural changes
#   across different macroeconomic periods.
#
# All analysis and visualization are conducted using Python and Plotly.
# This project demonstrates the use of reproducible code to analyze key
# macroeconomic indicators and evaluate the stability of a foundational
# empirical relationship in economics.
# ------------------------------------------------------------------------------


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [11]:
# Load economic data directly from a GitHub-hosted CSV file
def load_data():
    """
    Load time-series macroeconomic data (Unemployment and Inflation)
    from a public GitHub CSV file.

    Returns:
        pd.DataFrame: Cleaned DataFrame with datetime index and renamed columns
    """
    url = "https://raw.githubusercontent.com/yumnahussain444/phillips_curve_data/main/Copy%20of%20Copy%20of%20fredgraph.csv"
    df = pd.read_csv(url)

    # Convert observation_date to datetime
    df['observation_date'] = pd.to_datetime(df['observation_date'])

    # Rename FRED variable names to human-readable labels
    df = df.rename(columns={
        'UNRATE': 'Unemployment',
        'CPIAUCSL_PCH': 'Inflation'
    })

    # Set datetime as index for time-series analysis
    df.set_index('observation_date', inplace=True)

    return df

In [17]:
# Time Series Plotting

def create_time_series_plot(df):
    """
    Create a dual-axis time series plot of:
    - Unemployment Rate (left y-axis)
    - Inflation Rate (right y-axis)

    Args:
        df (pd.DataFrame): Data indexed by date with columns 'Unemployment' and 'Inflation'

    Returns:
        plotly.graph_objects.Figure: Interactive dual-axis plot
    """
    # Set up subplot layout with a secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add unemployment line to the primary y-axis (left)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['Unemployment'],
                   name="Unemployment Rate", line=dict(color="blue")),
        secondary_y=False
    )

    # Add inflation line to the secondary y-axis (right)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['Inflation'],
                   name="Inflation Rate", line=dict(color="red")),
        secondary_y=True
    )

    # Layout and visual styling
    fig.update_layout(
        title="Unemployment and Inflation Rates Over Time",
        xaxis_title="Date",
        yaxis_title="Unemployment Rate (%)",
        yaxis2_title="Inflation Rate (%)",
        showlegend=True,
        template="plotly_white"
    )

    return fig

# Generate and display the figure
df = load_data()
fig = create_time_series_plot(df)
fig.show()


In [21]:
#Scatter Plot of Inflation vs. Unemployment (Phillips Curve)

def create_scatter_plot(df):
    """
    Generate a scatter plot showing the Phillips Curve:
    - X-axis: Unemployment Rate
    - Y-axis: Inflation Rate
    - Includes OLS regression line and correlation annotation

    Args:
        df (pd.DataFrame): DataFrame with unemployment and inflation columns

    Returns:
        plotly.express.Figure: Scatter plot with trendline
    """
    # Create scatter with OLS trendline
    fig = px.scatter(
        df,
        x='Unemployment',
        y='Inflation',
        trendline="ols",
        title="Phillips Curve: Unemployment vs Inflation Rate",
        labels={'Unemployment': 'Unemployment Rate (%)',
                'Inflation': 'Inflation Rate (%)'},
        template="plotly_white"
    )

    # Compute and annotate correlation coefficient
    correlation = df['Unemployment'].corr(df['Inflation'])
    fig.add_annotation(
        text=f"Correlation: {correlation:.3f}",
        x=0.95, y=0.95, xref="paper", yref="paper",
        showarrow=False, font=dict(size=12)
    )

    return fig

# Generate and display scatter plot
fig = create_scatter_plot(df)
fig.show()

# Save as HTML
fig.write_html("phillips_curve.html")


In [22]:
# Rolling Correlation between Unemployment and Inflation

def generate_summary_statistics(df):
    """
    Generate summary statistics and compute rolling correlation.

    Args:
        df (pd.DataFrame): Input macroeconomic data

    Returns:
        tuple:
            - Summary statistics DataFrame
            - Overall correlation (float)
            - 5-year rolling correlation (Series)
    """
    # Descriptive stats for each variable
    summary_stats = df.describe()

    # Single correlation over the full sample
    correlation = df['Unemployment'].corr(df['Inflation'])

    # 5-year (60-month) rolling correlation
    rolling_corr = df['Unemployment'].rolling(window=60).corr(df['Inflation'])

    return summary_stats, correlation, rolling_corr

# ⏬ Compute and print stats
summary_stats, correlation, rolling_corr = generate_summary_statistics(df)
print("\nSummary Statistics:")
print(summary_stats)
print(f"\nFull-Sample Correlation: {correlation:.3f}")

# Create and show rolling correlation time series
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=rolling_corr, name='5-Year Rolling Correlation'))

fig.update_layout(
    title='5-Year Rolling Correlation between Unemployment and Inflation',
    xaxis_title='Date',
    yaxis_title='Correlation Coefficient',
    template="plotly_white"
)

fig.show()
fig.write_html("rolling_correlation.html")



Summary Statistics:
       Unemployment   Inflation
count    921.000000  931.000000
mean       5.688056    0.288292
std        1.709770    0.343765
min        2.500000   -1.800000
25%        4.400000    0.100000
50%        5.500000    0.200000
75%        6.700000    0.400000
max       14.800000    2.000000

Full-Sample Correlation: 0.007


In [20]:
#Phillips Curve by Decade (Faceted Subplots with Trendlines)

def analyze_phillips_curve_periods(df):
    """
    Generate decade-wise scatter plots of inflation vs. unemployment
    with OLS trendlines to analyze shifts in the Phillips Curve over time.

    Returns:
        plotly.express.Figure: Faceted grid of plots by decade
    """
    # Clean and filter data
    df = df.copy().dropna(subset=['Unemployment', 'Inflation'])
    df = df[df.index.year >= 1950]  # Focus on post-WWII data

    # Create a decade column (e.g., 1980, 1990)
    df['Decade'] = (df.index.year // 10) * 10
    decade_order = sorted(df['Decade'].unique())  # For ordered layout

    # Create faceted scatter plot with trendlines
    fig = px.scatter(
        df,
        x='Unemployment',
        y='Inflation',
        color='Decade',
        trendline='ols',
        facet_col='Decade',
        facet_col_wrap=5,
        category_orders={"Decade": decade_order},
        title="Phillips Curve by Decade (1950–Present)",
        labels={
            'Unemployment': 'Unemployment Rate (%)',
            'Inflation': 'Inflation Rate (%)'
        },
        template='plotly_white'
    )

    # Standardize axis ranges for comparability
    fig.update_yaxes(matches='y', range=[-2, 8])
    fig.update_xaxes(matches='x', range=[3, 12])

    # Final layout tweaks
    fig.update_layout(
        height=600,
        margin=dict(t=60, l=40, r=40, b=40),
        title_font_size=18
    )

    return fig

# Generate and show figure
fig = analyze_phillips_curve_periods(df)
fig.show()
fig.write_html("phillips_curve_by_decade.html")
